In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import langdetect
from time import sleep
from random import randint
import json

In [3]:
f = open('beers_to_scrape.json')
dict_beers = json.load(f)

In [ ]:
page = requests.get("https://www.ratebeer.com/beer/sierra-nevada-pale-ale-bottle-can/365/")

soup = bs(page.content, "html.parser")

beer_table = soup.find_all("dic", {"class": "LinesEllipsis  "})

In [5]:
from selenium import webdriver

DRIVER_PATH = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.ratebeer.com/beer/sierra-nevada-pale-ale-bottle-can/365/')
driver.implicitly_wait(7)
driver.execute_script("return document.documentElement.outerHTML")

print(driver.page_source)

#getting the button by class name
button = driver.
button.click()

<html lang="en"><head><!--mui-inject-first--><style type="text/css">svg:not(:root).svg-inline--fa {
  overflow: visible;
}

.svg-inline--fa {
  display: inline-block;
  font-size: inherit;
  height: 1em;
  overflow: visible;
  vertical-align: -0.125em;
}
.svg-inline--fa.fa-lg {
  vertical-align: -0.225em;
}
.svg-inline--fa.fa-w-1 {
  width: 0.0625em;
}
.svg-inline--fa.fa-w-2 {
  width: 0.125em;
}
.svg-inline--fa.fa-w-3 {
  width: 0.1875em;
}
.svg-inline--fa.fa-w-4 {
  width: 0.25em;
}
.svg-inline--fa.fa-w-5 {
  width: 0.3125em;
}
.svg-inline--fa.fa-w-6 {
  width: 0.375em;
}
.svg-inline--fa.fa-w-7 {
  width: 0.4375em;
}
.svg-inline--fa.fa-w-8 {
  width: 0.5em;
}
.svg-inline--fa.fa-w-9 {
  width: 0.5625em;
}
.svg-inline--fa.fa-w-10 {
  width: 0.625em;
}
.svg-inline--fa.fa-w-11 {
  width: 0.6875em;
}
.svg-inline--fa.fa-w-12 {
  width: 0.75em;
}
.svg-inline--fa.fa-w-13 {
  width: 0.8125em;
}
.svg-inline--fa.fa-w-14 {
  width: 0.875em;
}
.svg-inline--fa.fa-w-15 {
  width: 0.9375em;
}
.svg-i